In [43]:
#Import packages
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import shutil
import os
import time

In [44]:
#Initialize global variables for path & date
TodaysDate = time.strftime("%#m.%d.%Y")
dl_path = 'C:/Users/vorbej1/Downloads/'
share_dr_path = 'X:/Ying_Ba/MH - Safety Metrics/IMS Reports/'

In [45]:
#Load WebDriver
driver = webdriver.Chrome('C:/Users/vorbej1/desktop/chromedriver.exe')

In [46]:
#Load IMS website - incident metrics
driver.get('http://nyhixapp91v/GlobalIMS/Incidents/IncidentAdvSearch_V3.aspx')

In [47]:
#Select Dates
date_from = driver.find_element_by_id("ctl00_ContentPlaceHolder1_tbxdtFrom")
date_from.clear()
date_from.send_keys('4/1/2019')

In [48]:
#Select near-misses & good catches
driver.find_element_by_id("ctl00_ContentPlaceHolder1_cbxNM").click()
driver.find_element_by_id("ctl00_ContentPlaceHolder1_cbxHaz").click()

In [49]:
#Select National Grid
driver.find_element_by_id("chk_US").click()

In [50]:
#Select drop-down
driver.find_element_by_xpath('//img[@src="../tree2/style/web/plusik_l.gif"]').click()

In [51]:
#Select gas operations for gas ops incidents
driver.find_element_by_id('chk_A_US_97500242').click()

In [52]:
#Select incident details to add additional information
driver.find_element_by_link_text('Incident Details').click()

In [53]:
time.sleep(3)

In [54]:
#Include additional incident information - 8:Reported by, 9:Created by, 13:Incident descr., 23:Employee id, 27:Union
additional_inc_ids = ['8','9','13','23','27']

for inc_id in additional_inc_ids:
    driver.find_element_by_id("ctl00_ContentPlaceHolder1_pnlColumns_i6_i" + inc_id + "_cbx_SelectAsOutput").click()

In [55]:
#Select organization 
driver.find_element_by_link_text('Organization').click()
time.sleep(.5)

In [56]:
#Check all organizations within gas ops
org_select = driver.find_elements_by_xpath('//ul[@class="rpRootGroup"]/li[9]//input[@type="checkbox"]')

for orgs in org_select:
    orgs.click()

In [57]:
#Search
driver.find_element_by_id('ctl00_ContentPlaceHolder1_btn_Search').click()

In [58]:
#Allow time for table to load
time.sleep(5)

In [59]:
#Obtain number of rows for report to use to split array into rows for dataframe
#Number of rows is printed with the report and will be dynamic as more records are added
n_rows = driver.find_element_by_xpath('//span[@id="ctl00_ContentPlaceHolder1_lblRecords"]').text
n_rows = int(n_rows[0:3])

In [60]:
#Extract column headers
for th in driver.find_elements_by_xpath('//table[@id="ctl00_ContentPlaceHolder1_GridView1"]'):
        ths = th.find_elements_by_tag_name('th')
        cols = [th.text for th in ths]

In [61]:
#Extract table rows
for td in driver.find_elements_by_xpath('//table[@id="ctl00_ContentPlaceHolder1_GridView1"]'):
    tds = td.find_elements_by_tag_name('td')
    data = [td.text for td in tds]

In [62]:
#Create dataframe - assign extracted column headers for colnames and split array into # of rows
incidents_report = pd.DataFrame(columns=cols, data=np.array_split(data , n_rows))

In [63]:
#Write to excel for alteryx input
incidents_report.to_excel('X:/Ying_Ba/MH - Safety Metrics/Input/incidents_report.xls')